In [1]:
!pip install iterative-stratification

You should consider upgrading via the 'c:\users\clive\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


# Multi-label text classification with sklearn

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score,hamming_loss,accuracy_score

import os
print(os.listdir())

%matplotlib inline

['.ipynb_checkpoints', 'df_questions', 'StackOverFlowQuestions.ipynb', 'Tags.csv']


In [2]:
# start from here
import pickle as pk

df_questions = pk.load(open('df_questions','rb'))

df_questions.head()



,Id,OwnerUserId,CreationDate,Score,Title,Body,Text,Tags,procesed_text
0,6,5.0,2010-07-19T19:14:44Z,272,The Two Cultures: statistics vs. machine learn...,"Last year, I read a blog post from Brendan O'C...",The Two Cultures: statistics vs. machine learn...,[machine-learning],the two cultur statist machin learn last year...
1,21,59.0,2010-07-19T19:24:36Z,4,Forecasting demographic census,What are some of the ways to forecast demograp...,Forecasting demographic census What are some o...,[forecasting],forecast demograph censu what way forecast de...
2,22,66.0,2010-07-19T19:25:39Z,208,Bayesian and frequentist reasoning in plain En...,How would you describe in plain English the ch...,Bayesian and frequentist reasoning in plain En...,[bayesian],bayesian frequentist reason plain english how...
3,31,13.0,2010-07-19T19:28:44Z,138,What is the meaning of p values and t values i...,After taking a statistics course and then tryi...,What is the meaning of p values and t values i...,"[hypothesis-testing, t-test, p-value, interpre...",what mean valu valu statist test after take s...
4,36,8.0,2010-07-19T19:31:47Z,58,Examples for teaching: Correlation does not me...,"There is an old saying: ""Correlation does not ...",Examples for teaching: Correlation does not me...,[correlation],exampl teach correl causat there correl causa...


# Post Processing 


1. when word len < 2 is removed it produces the best results with accuracy of 50 for train and 48 for test 
2. when word len < 3 is remove the accuracy reduces to train acc. = 48 and test acc = 45
3. when word len < 4 is removed the accuracy reduces to train acc = 46 and test acc = 44

In [3]:
# need to do post processing 

counter_words2remove = 0
counter_total_words = 0
for text in df_questions['procesed_text']:

  for words in text.split():

    if len(words) <= 2:
      counter_words2remove += 1

    counter_total_words += 1  

print(counter_words2remove /  counter_total_words)      

0.2715669351211727


In [4]:

def post_process(x):
  ls = [] 
  for word in x.split():

      if len(word) >2:
        ls.append(word)

  return ' '.join(ls)



df_questions['post_processed_text'] = df_questions['procesed_text'].map(post_process)

#printing the word reduction
counter_total_words = 0

for text in df_questions['procesed_text']:

  for words in text.split():

    counter_total_words += 1  

print('no of words',counter_total_words)

counter_total_words = 0

for text in df_questions['post_processed_text']:

  for words in text.split():


    counter_total_words += 1  

print('no of words',counter_total_words)

no of words 8446993
no of words 6153069


# Label Encoding

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer


# for labels 
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_questions.Tags)
Y = multilabel_binarizer.transform(df_questions.Tags)



In [96]:
# import tqdm 

# def is_same(x1,x2):
    
#     return sum(x1 == x2) == 100



# for index,samples in tqdm.tqdm(enumerate(Y)):
    
#     count = []
#     indexs = []
#     for idx,x1 in enumerate(Y):
        
#         if is_same(x1,samples) == True:
#             indexs.append(idx)
#             count.append(True)
    
    
#     print(f'sample {index + 1} is has {sum(count)-1} copies')
#     print(indexs)
    
#     break
    
    

In [6]:
#calculating class weights

df_tags = pd.read_csv('Tags.csv')

num_classes = 100
grouped_tags = df_tags.groupby("Tag").size().reset_index(name='count')
most_common_tags = grouped_tags.nlargest(num_classes, columns="count")
df_tags.Tag = df_tags.Tag.apply(lambda tag : tag if tag in most_common_tags.Tag.values else None)

df_tags = df_tags.dropna()


In [144]:
total_count = most_common_tags['count'].sum()
print(total_count)

def get_class_weight(x):

    return 1 / (x /  total_count)


most_common_tags['class_weight'] = most_common_tags['count'].map(get_class_weight)

most_common_tags

152913


,Tag,count,class_weight,class_id
0,r,13236,11.552811,74
1,regression,10959,13.953189,79
2,machine-learning,6089,25.112991,41
3,time-series,5559,27.507285,98
4,probability,4217,36.261086,71
...,...,...,...,...
95,nonlinear-regression,514,297.496109,58
96,cox-model,510,299.829412,17
97,monte-carlo,504,303.398810,52
98,proportion,503,304.001988,72


In [149]:
class_weights = dict(most_common_tags[['Tag','class_weight']].values)

# Text Encoding 
# Text Encoding
1. Using (1,2),(2,2),(1,3), ngrams doesnt help 
2. using  use_idf=1,smooth_idf=1,sublinear_tf=True increase accuracy by 2%
3. using max features = 20000 helps reduce 
  the same effect can be observed using min_df = 2
      and max_df = 0.95 (meaning ignore terms that occur in less than 1 doucment and terms that occure in more than 95% of the documents.
4. max_features also help in reducing overfitting
5. processed_text has a accuracy of 48 
5. post_processed_text with n == 3 has a accuracy of 46

In [47]:
#tfidf 
# tfv = TfidfVectorizer(min_df=9,max_df=0.70,  max_features=None, 
#             strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
#             ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
#             stop_words = 'english')

#remove words that have appeat in less than 2 documents and appear in more that 95 % 

tfv = TfidfVectorizer(use_idf=1,smooth_idf=1,sublinear_tf=True,ngram_range = (1,1),
#                       min_df = 2,max_df = 0.95,
                      max_features = 50000
                      
                     )
words = tfv.fit(df_questions['post_processed_text'])
X = tfv.transform(df_questions['post_processed_text'])

In [14]:
X

<76365x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 3410862 stored elements in Compressed Sparse Row format>

In [103]:
Y.shape

(76365, 100)

In [8]:
# #
# from sklearn.utils.class_weight import compute_sample_weight


# sample_weights = compute_sample_weight(class_weight='balanced', y=Y)

# Splitting Train Validation split

In [15]:
from sklearn.model_selection import train_test_split



trainx,testx,trainy,testy = train_test_split(X,Y,test_size = 0.3,shuffle = True)

trainx.shape,trainy.shape

((53455, 50000), (53455, 100))

In [9]:
trainx

<53455x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 2637505 stored elements in Compressed Sparse Row format>

# ML model
## SGDClassifier with loss = 'log' 
(using logistic regression)

SGDClassifier?
hing loss give 30 acc
log with l1  38 acc
log with l1 svd 45 val acc and 53 train acc (ignore this one)







In [18]:
#include 
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
import time

t1 = time.time()

estimator = SGDClassifier(
                          alpha = 0.00002,
                          n_jobs= -1,
                          random_state = 909,
                          loss = 'log',
                          penalty = 'l1',
#                           learning_rate = 'invscaling',
#                           eta0 = 500,
#                           power_t = 0.5,
                          
                          shuffle = True,
                          early_stopping=True,
                          
                         )
clf  = OneVsRestClassifier(estimator,n_jobs= -1)

clf.fit(trainx,trainy)


print(int(time.time() - t1))


10


Evaluating the Score

In [19]:
from sklearn.metrics import f1_score,hamming_loss,accuracy_score

pred_train = clf.predict(trainx)
pred_test = clf.predict(testx)

# pred_test_proba = clf.predict_proba(testx)


print('train loss:',hamming_loss(trainy,pred_train))
print('test loss:',hamming_loss(testy,pred_test))


print('train score:',f1_score(trainy,pred_train,average  = 'micro'))
print('test score:',f1_score(testy,pred_test,average = 'micro'))

train loss: 0.015188289215227762
test loss: 0.015897861195984287
train score: 0.4993370867583851
test score: 0.476710439355191


In [58]:
# del clf
# del pred_test_proba
# del pred_train
# del pred_test
del X,Y,trainx,testx
import gc
gc.collect()

20

In [138]:
for i in range(100):
    
    print('true : ',multilabel_binarizer.inverse_transform(np.array([testy[i]])))
    print('pred : ',multilabel_binarizer.inverse_transform(np.array([pred_test[i]])))
    
    
    print("**********")

true :  [('anova',)]
pred :  [('hypothesis-testing', 't-test')]
**********
true :  [('bayesian', 'conditional-probability', 'probability')]
pred :  [('conditional-probability', 'probability')]
**********
true :  [('machine-learning', 'mathematical-statistics', 'sampling', 'statistical-significance')]
pred :  [()]
**********
true :  [('data-visualization',)]
pred :  [('data-visualization',)]
**********
true :  [('r', 'regression')]
pred :  [('generalized-linear-model', 'r')]
**********
true :  [('statistical-significance',)]
pred :  [('statistical-significance',)]
**********
true :  [('feature-selection', 'regression', 'residuals')]
pred :  [('regression',)]
**********
true :  [('econometrics', 'self-study')]
pred :  [()]
**********
true :  [('correlation', 'r', 'time-series')]
pred :  [('regression',)]
**********
true :  [('regression',)]
pred :  [('regression',)]
**********
true :  [('regression', 'regression-coefficients', 'self-study')]
pred :  [('regression',)]
**********
true :  [

In [139]:
from sklearn.metrics import classification_report
import pprint
pprint.pprint(classification_report(testy,pred_test))


('              precision    recall  f1-score   support\n'
 '\n'
 '           0       0.27      0.02      0.04       143\n'
 '           1       0.80      0.57      0.67       797\n'
 '           2       0.72      0.54      0.62       251\n'
 '           3       0.74      0.45      0.56       206\n'
 '           4       0.77      0.61      0.68       767\n'
 '           5       0.40      0.19      0.26       157\n'
 '           6       0.70      0.35      0.47       293\n'
 '           7       0.88      0.75      0.81       236\n'
 '           8       0.70      0.34      0.46       182\n'
 '           9       0.58      0.22      0.32       510\n'
 '          10       0.79      0.55      0.65       404\n'
 '          11       0.64      0.38      0.47       854\n'
 '          12       0.84      0.66      0.74       571\n'
 '          13       0.61      0.12      0.21       272\n'
 '          14       0.73      0.50      0.60       509\n'
 '          15       0.70      0.51      0.59     

c:\users\clive\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [140]:
#using different method of train test split.

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

mskf = MultilabelStratifiedKFold(shuffle = True,random_state = 909,n_splits = 3)

estimator = SGDClassifier(n_jobs= -1,
                          random_state = 909,
                          loss = 'log',
                          penalty = 'l1',
#                           l1_ratio = 0.5,
                    
                          shuffle = False,
                          early_stopping=True)

clf  = OneVsRestClassifier(estimator,n_jobs= -1)


for train_index, test_index in mskf.split(X_std, Y):
#    print("TRAIN:", train_index, "TEST:", test_index)

     trainx,testx = X_std[train_index],X_std[test_index]
     trainy,testy = Y[train_index],Y[test_index] 
        

     clf.fit(trainx,trainy)
        
        
     pred_train = clf.predict(trainx)
     pred_test = clf.predict(testx)

     print('train loss:',hamming_loss(trainy,pred_train))
     print('test loss:',hamming_loss(testy,pred_test))


     print('train score:',f1_score(trainy,pred_train,average  = 'micro'))
     print('test score:',f1_score(testy,pred_test,average = 'micro'))   
        


c:\users\clive\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass shuffle=True, random_state=909 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


train loss: 0.01814390071479067
test loss: 0.020321528241814393
train score: 0.48119532381775915
test score: 0.4272864232541652
train loss: 0.018249990164063423
test loss: 0.020386980533469115
train score: 0.47961587650609167
test score: 0.4283863032353006
train loss: 0.018166444322373066
test loss: 0.020504469735753948
train score: 0.47899036730284483
test score: 0.4234189340331993



## SGDClassifier with loss = hinge (SVM)

In [20]:
#include 
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

t1 = time.time()



estimator = SGDClassifier(n_jobs= -1)

clf  = OneVsRestClassifier(estimator,n_jobs= -1)

clf.fit(trainx,trainy)


pred_train = clf.predict(trainx)
pred_test = clf.predict(testx)


print(time.time() - t1)

print('train loss:',hamming_loss(trainy,pred_train))
print('test loss:',hamming_loss(testy,pred_test))

# print('train score:',accuracy_score(trainy,pred_train))
# print('test score:',accuracy_score(testy,pred_test))

print('train score:',f1_score(trainy,pred_train,average  = 'micro'))
print('test score:',f1_score(testy,pred_test,average = 'micro'))


print('train score:',f1_score(trainy,pred_train,average  = 'macro'))
print('test score:',f1_score(testy,pred_test,average = 'macro'))



6.547489166259766
train loss: 0.015608081563932279
test loss: 0.016456569183762548
train score: 0.41289432759360767
test score: 0.37297099521021826
train score: 0.25845991352825093
test score: 0.22647095028689115


# Using loss = modified_huber

In [21]:
#include 
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier


t1 = time.time()

estimator = SGDClassifier(n_jobs= -1,loss = 'modified_huber',penalty = 'l1',l1_ratio=0.150)

clf  = OneVsRestClassifier(estimator,n_jobs= -1)

clf.fit(trainx,trainy)

print(time.time() - t1)



from sklearn.metrics import f1_score,hamming_loss,accuracy_score

pred_train = clf.predict(trainx)
pred_test = clf.predict(testx)




print('train loss:',hamming_loss(trainy,pred_train))
print('test loss:',hamming_loss(testy,pred_test))

print('train score :',1 - hamming_loss(trainy,pred_train))
print('test score :',1 - hamming_loss(testy,pred_test))



# print('train score:',accuracy_score(trainy,pred_train))
# print('test score:',accuracy_score(testy,pred_test))

print('train score:',f1_score(trainy,pred_train,average  = 'micro'))
print('test score:',f1_score(testy,pred_test,average = 'micro'))


for i in range(100):
    
    print('true : ',multilabel_binarizer.inverse_transform(np.array([testy[i]])))
    print('pred : ',multilabel_binarizer.inverse_transform(np.array([pred_test[i]])))
    
    
    print("**********")


10.406828165054321
train loss: 0.015546721541483491
test loss: 0.016058489742470535
train score : 0.9844532784585165
test score : 0.9839415102575295
train score: 0.46370377062615753
test score: 0.44471277205904547
true :  [('deep-learning', 'machine-learning')]
pred :  [()]
**********
true :  [('machine-learning', 'neural-networks', 'terminology')]
pred :  [('neural-networks',)]
**********
true :  [('correlation', 'hypothesis-testing')]
pred :  [()]
**********
true :  [('r', 'time-series')]
pred :  [('forecasting', 'r')]
**********
true :  [('maximum-likelihood', 'self-study')]
pred :  [()]
**********
true :  [('standard-deviation',)]
pred :  [('standard-deviation',)]
**********
true :  [('machine-learning', 'neural-networks', 'r')]
pred :  [('r',)]
**********
true :  [('regression',)]
pred :  [('regression',)]
**********
true :  [('panel-data',)]
pred :  [()]
**********
true :  [('correlation', 'expected-value', 'mathematical-statistics', 'probability')]
pred :  [('covariance',)]
****

In [26]:
import pandas as pd


'''
 0.41289432759360767
test score: 0.37297099521021826


train score: 0.46370377062615753
test score: 0.44471277205904547
'''

data = {
        'loss':['log','hinged','modfied_huber'],
        'train_acc':[0.499,0.4128,0.4637],
        'val_acc':[0.4767,0.3729,0.4447],
        'train_time':[10,6,10.40]
        }


pd.DataFrame(data)



,loss,train_acc,val_acc,train_time
0,log,0.4990,0.4767,10.0
1,hinged,0.4128,0.3729,6.0
2,modfied_huber,0.4637,0.4447,10.4


In [28]:
0.5 - 0.48

0.020000000000000018

#### using log loss has highest train and val acc and lowes accuracy of abt 2%  the training time is also 
#### moderate 10s is good for 85000 samples

# Using Truncated_SVD

(n_components = 400)
1. with tfidf + svd => train - 47 and test - 46 traning time is 2min
2. with tfidf + svd + standard_scalar => train -45 and test 43 overfitting 
3. with tfidf + standard_scalar + svd = > train -12 and test- 11


In [48]:
# tfv = TfidfVectorizer(use_idf=1,smooth_idf=1,sublinear_tf=True,ngram_range = (1,1),
# #                       min_df = 2,max_df = 0.95,
#                       max_features = 50000
                      
#                      )
# words = tfv.fit(df_questions['procesed_text'])
# X = tfv.transform(df_questions['procesed_text'])

##### Using processed reduces the accuracy and causes more overfitting

In [49]:
from sklearn.decomposition import TruncatedSVD,PCA

t1 = time.time()


tsvd = TruncatedSVD(n_components = 400)

X_svd = tsvd.fit_transform(X)


print(time.time() - t1)


82.9757559299469


In [50]:
from sklearn.model_selection import train_test_split

trainx,testx,trainy,testy = train_test_split(X_svd,Y,test_size = 0.3)

trainx.shape,trainy.shape

((53455, 400), (53455, 100))

# logistic regression

In [51]:
#include 
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier


t1 = time.time()

estimator = SGDClassifier(alpha = 0.00002,
                          n_jobs= -1,
                          random_state = 909,
                          loss = 'log',
                          penalty = 'l1',
#                           learning_rate = 'invscaling',
#                           eta0 = 500,
#                           power_t = 0.5,
                          
                          shuffle = True,
                          early_stopping=True,
                          
                         )

clf  = OneVsRestClassifier(estimator,n_jobs= -1)

clf.fit(trainx,trainy)

print(time.time() - t1)



76.64832544326782


In [33]:
75 + 76

151

In [52]:
from sklearn.metrics import f1_score,hamming_loss,accuracy_score

pred_train = clf.predict(trainx)
pred_test = clf.predict(testx)

print('train loss:',hamming_loss(trainy,pred_train))
print('test loss:',hamming_loss(testy,pred_test))

# print('train score:',accuracy_score(trainy,pred_train))
# print('test score:',accuracy_score(testy,pred_test))

print('train score:',f1_score(trainy,pred_train,average  = 'micro'))
print('test score:',f1_score(testy,pred_test,average = 'micro'))

train loss: 0.01582059676363296
test loss: 0.016168485377564382
train score: 0.480735341974543
test score: 0.4652827900799723


##### training the model with svd takes the most time > 2m 
##### the train acc and val accuracy is reduced.
##### overfitting is also reduced from 2% to 1%

# tfidf +  std

In [53]:
from sklearn.preprocessing import StandardScaler


t1 = time.time()


X_std = StandardScaler(with_mean = False).fit_transform(X)


# tsvd = TruncatedSVD(n_components = 400)

# X_svd = tsvd.fit_transform(X_std)


trainx,testx,trainy,testy = train_test_split(X_std,Y,test_size = 0.3)


#include 
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

estimator = SGDClassifier(alpha = 0.002,
                          n_jobs= -1,
                          random_state = 909,
                          loss = 'log',
                          penalty = 'l1',
#                           learning_rate = 'invscaling',
#                           eta0 = 500,
#                           power_t = 0.5,
                          
                          shuffle = True,
                          early_stopping=True,
                          
                         )

clf  = OneVsRestClassifier(estimator,n_jobs= -1)

clf.fit(trainx,trainy)


print(time.time() - t1)

pred_train = clf.predict(trainx)
pred_test = clf.predict(testx)

print('train loss:',hamming_loss(trainy,pred_train))
print('test loss:',hamming_loss(testy,pred_test))

# print('train score:',accuracy_score(trainy,pred_train))
# print('test score:',accuracy_score(testy,pred_test))

print('train score:',f1_score(trainy,pred_train,average  = 'micro'))
print('test score:',f1_score(testy,pred_test,average = 'micro'))




27.202172994613647
train loss: 0.02214367224768497
test loss: 0.028030990833697075
train score: 0.4752658713798714
test score: 0.3599796689223532


In [36]:
75 + 35

110

##### by std scaling the svd matrix we get more overfitting of about 12% 
##### the train and val acc is also low.

# tfidf + std + svd

In [54]:
from sklearn.preprocessing import StandardScaler


t1 = time.time()


X_std = StandardScaler(with_mean = False).fit_transform(X)


tsvd = TruncatedSVD(n_components = 400)

X_svd = tsvd.fit_transform(X_std)


trainx,testx,trainy,testy = train_test_split(X_svd,Y,test_size = 0.3)


#include 
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

estimator = SGDClassifier(alpha = 0.002,
                          n_jobs= -1,
                          random_state = 909,
                          loss = 'log',
                          penalty = 'l1',
#                           learning_rate = 'invscaling',
#                           eta0 = 500,
#                           power_t = 0.5,
                          
                          shuffle = True,
                          early_stopping=True,
                          
                         )

clf  = OneVsRestClassifier(estimator,n_jobs= -1)

clf.fit(trainx,trainy)


print(time.time() - t1)

pred_train = clf.predict(trainx)
pred_test = clf.predict(testx)

print('train loss:',hamming_loss(trainy,pred_train))
print('test loss:',hamming_loss(testy,pred_test))

# print('train score:',accuracy_score(trainy,pred_train))
# print('test score:',accuracy_score(testy,pred_test))

print('train score:',f1_score(trainy,pred_train,average  = 'micro'))
print('test score:',f1_score(testy,pred_test,average = 'micro'))




138.86205554008484
train loss: 0.03660462070900758
test loss: 0.03835573985159319
train score: 0.11647039699454538
test score: 0.10972310872009969


with tfidf + svd + standard_scalar => train -52 and test 45 overfitting 

In [55]:
from sklearn.preprocessing import StandardScaler


t1 = time.time()

tsvd = TruncatedSVD(n_components = 400)

X_svd = tsvd.fit_transform(X)


X_std = StandardScaler(with_mean = False).fit_transform(X_svd)





trainx,testx,trainy,testy = train_test_split(X_std,Y,test_size = 0.3)


#include 
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

estimator = SGDClassifier(alpha = 0.002,
                          n_jobs= -1,
                          random_state = 909,
                          loss = 'log',
                          penalty = 'l1',
#                           learning_rate = 'invscaling',
#                           eta0 = 500,
#                           power_t = 0.5,
                          
                          shuffle = True,
                          early_stopping=True,
                          
                         )

clf  = OneVsRestClassifier(estimator,n_jobs= -1)

clf.fit(trainx,trainy)


print(time.time() - t1)

pred_train = clf.predict(trainx)
pred_test = clf.predict(testx)

print('train loss:',hamming_loss(trainy,pred_train))
print('test loss:',hamming_loss(testy,pred_test))

# print('train score:',accuracy_score(trainy,pred_train))
# print('test score:',accuracy_score(testy,pred_test))

print('train score:',f1_score(trainy,pred_train,average  = 'micro'))
print('test score:',f1_score(testy,pred_test,average = 'micro'))




155.29935145378113
train loss: 0.019768029183425313
test loss: 0.020475338280226976
train score: 0.4581192373567993
test score: 0.43804731955675347


In [45]:
data = {'cofig':['tfidf+svd','tfidf+std','tfidf+std+svd','tfidf+svd+std'],
        'train_acc':[47,47,12,45],
        'val_acc':[46,35,11,43],
        'time(s)':[150,110,137,149]
       }

pd.DataFrame(data)

,cofig,train_acc,val_acc,time(s)
0,tfidf+svd,47,46,150
1,tfidf+std,47,35,110
2,tfidf+std+svd,12,11,137
3,tfidf+svd+std,45,43,149


#### using svd the overfitting is reduced but the time is increases 

# LinearSVM

In [44]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

estimator = LinearSVC(penalty = 'l1',dual = False)

clf2 = OneVsRestClassifier(estimator,n_jobs = -1)

clf2.fit(trainx,trainy)

from sklearn.metrics import f1_score,hamming_loss,accuracy_score

pred_train = clf2.predict(trainx)
pred_test = clf2.predict(testx)

print('train loss:',hamming_loss(trainy,pred_train))
print('test loss:',hamming_loss(testy,pred_test))

# print('train score:',accuracy_score(trainy,pred_train))
# print('test score:',accuracy_score(testy,pred_test))

print('train score:',f1_score(trainy,pred_train,average  = 'micro'))
print('test score:',f1_score(testy,pred_test,average = 'micro'))




train loss: 0.01591600411561126
test loss: 0.01664687909209952
train score: 0.4382852577857299
test score: 0.40764786282306165


# Best Model


1. loss = 'log'
2. alpha  = 0.0002
3. penalty = 'l1'
4. early_stopping = True
5. shuffle = True
6. Using only Tfidf with max_features = 50000 (if max. features is reduced it causes overfitting.)
7. Using procesed_text and 
8. post_processed_test with wordlen = 2
9. both 7 and 8 produce same results with 7 producing slightly higher results 






# Traning Time Comparision

### SGDClassifeir Tunning 
#### # of parameters
1. loss
2. penalty *
3. alpha **
4. l1_ratio
5. epsilon
6. class_weight

7. learning rate = 'constant' , 'optimal','invscaling','adaptive'
7.1 eta0
7.2 power_t

8. average

9. early_stopping 

10. validation_fraction 
    The proportion of training data to set aside as validation set for early stopping. Must be between 0 and 1. Only used if early_stopping is True.

11. n_iter_no_changeint, default=5

    Number of iterations with no improvement to wait before early stopping.    